In [1]:
import pandas as pd
import csv

df = pd.read_csv('training_data_w_ingredients.csv', sep=';', error_bad_lines=False)
df.head(20)

,Unnamed: 0,index,name,ingredients,cuisine
0,1,1,Vietnamese Pho Soup,"['8 cups beef broth', '4 cups water', '1 yello...",Vietnamese
1,2,2,Vietnamese Chicken Meatballs,"['1 pound ground chicken', '3 tablespoons fish...",Vietnamese
2,3,3,Vietnamese Restaurant-Style Grilled Lemongrass...,"['1 pound pork blade steaks (boneless, about 1...",Vietnamese
3,4,4,Vietnamese Caramel Chicken,"['1 pound chicken thighs (with skin, deboned)'...",Vietnamese
4,5,5,Cheater Vietnamese Pho (Pho Bo),"['3 marrow bones (beef bone)', '50 ounces beef...",Vietnamese
5,6,6,Authentic Vietnamese Beef Pho (Pho Bo),['3 pounds beef (knuckles or neck bones with m...,Vietnamese
6,7,7,Vietnamese Pho Recipe (Vietnamese Beef Noodle ...,['600 grams onions (about 3 to 4 medium sized ...,Vietnamese
7,8,8,Pho {Vietnamese Beef Noodle Soup},"['2 tablespoons coriander seeds (whole)', '1 c...",Vietnamese
8,9,9,Vietnamese Spring Rolls,"['12 spring rolls (wrappers)', '18 cooked shri...",Vietnamese
9,10,10,Fresh and Easy Vietnamese Noodle Salad,"['12 ounces vermicelli noodles (thin Asian, su...",Vietnamese


## Ingredients, list to string

For every row we add a column called Ingredients_Str which contains the concatinated Strings from the Ingredients column


## Text Processing 
Now that the data is retrieved, I need to process the data for the model to train on. I use the work from my local Jupyter notebook and make that work here in Sagemaker.

## Writing a Custom Transformer 
When working locally on the data, I did not perform any predictions as I was using the data I had scraped, divided it into train/test to determine the model performance. Since I will be using the SKLearn pipeline for implementation and I plan to use it for predicting on new data (in the API endpoint), I need to format the parse_recipes function to use in a scikit-learn pipeline. It is not necessary but I would rather have it all in one pipeline.

FunctionTransformer (simpler functions), Fixing for state

A FunctionTransformer class helps to introduce arbitrary, stateless transforms into a Pipeline. Using a TransformerMixin class is overkill for what I need to do, though since I was able to get it to work with the recipe data, I am using it for now.

In [16]:
import string
from sklearn.base import TransformerMixin

class TransformRecipe(TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y):
        return self
    def transform(self,X, **kwargs):
        recipe_list=X
        ingr_list=[]
        bigram_list=[]
        recipe_string_list=[]

#         ilist = [[word.lower() for word in i.split()] for i in ilist] 
        for recipe in recipe_list:
#             print(recipe)
            ingr_list=[]
            for ingredient in recipe:

                ingredient.translate(str.maketrans('', '', string.punctuation))

                words = ingredient.split()
                words = [''.join(c for c in word if c not in string.punctuation) for word in words]
                words = [word for word in words if word.isalpha()]
                words = [word.lower() for word in words] 
                words = [lemmatizer.lemmatize(word) for word in words]
                words = [word for word in words if word not in measures]
                words = [word for word in words if word not in common_remove]
                words = [word for word in words if word not in data_leaks]
                #get rid of any blank
                words = list(filter(None, words))
#                 print("before if length statements")
#                 print(words)
                if(len(words)<=3):
                    ingr_list.append(' '.join(words))

                words = [word for word in words if word not in useless_singles]
            #easiest way to deal with any duplicates or blanks for now
                ingr_list = list(set(ingr_list))
                #attempts to get rid of the blank
#                 ingr_list = list(filter(None, ingr_list))
#                 ingr_list = [x for x in ingr_list if x]   
                if(len(words)>3): #handle rare case
                    ingr_list.append(' '.join(words))
                recipe_string=' '.join(ingr_list)
#             print(recipe_string)

            recipe_string_list.append(recipe_string)

        return recipe_string_list

## Words to Remove 
This code and these word lists are copied over as is.

In [17]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

#Source for list below
#https://en.wikipedia.org/wiki/Cooking_weights_and_measures
#https://thebakingpan.com/ingredient-weights-and-measures/
measures=['litrbes','liter','millilitres','mL','grams','g', 'kg','teaspoon','tsp', 'tablespoon','tbsp','fluid', 'ounce','oz','fl.oz', 'cup','pint','pt','quart','qt','gallon','gal','smidgen','drop','pinch','dash','scruple','dessertspoon','teacup','cup','c','pottle','gill','dram','wineglass','coffeespoon','pound','pounded','lb','tbsp','plus','firmly', 'packed','lightly','level','even','rounded','heaping','heaped','sifted','bushel','peck','stick','chopped','sliced','halves', 'shredded','slivered','sliced','whole','paste','whole',' fresh', 'peeled', 'diced','mashed','dried','frozen','fresh','peeled','candied','no', 'pulp','crystallized','canned','crushed','minced','julienned','clove','head', 'small','large','medium', 'torn', 'cleaned', 'degree']

measures = [lemmatizer.lemmatize(m) for m in measures]
#some of these include data leakage words, like 'italian' - ok to remove after including bigrams
data_leaks = ['iberic','greek', 'korean','italianstyle', 'french','thai', 'chinese', 'mexican','spanish','indian','italian']

common_remove=['ground','to','taste', 'and', 'or',  'can',  'into', 'cut', 'grated', 'leaf','package','finely','divided','a','piece','optional','inch','needed','more','drained','for','flake','dry','thinly','cubed','bunch','cube','slice','pod','beaten','seeded','uncooked','root','plain','heavy','halved','crumbled','sweet','with','hot','room','temperature','trimmed','allpurpose','deveined','bulk','seasoning','jar','food','if','bag','mix','in','each','roll','instant','double','such','frying','thawed','whipping','stock','rinsed','mild','sprig','freshly','toasted','link','boiling','cooked','unsalted','container',
'cooking','thin','lengthwise','warm','softened','thick','quartered','juiced','pitted','chunk','melted','cold','coloring','puree','cored','stewed','floret','coarsely','the','blanched','zested','sweetened','powdered','garnish','dressing','soup','at','active','lean','chip','sour','long','ripe','skinned','fillet','from','stem','flaked','removed','stalk','unsweetened','cover','crust', 'extra', 'prepared', 'blend', 'of', 'ring',  'undrained', 'about', 'zest', ' ', '', 'spray', 'round', 'herb', 'seasoned', 'wedge', 'bitesize', 'broken', 'square', 'freshly', 'thickly', 'diagonally']
common_remove = [lemmatizer.lemmatize(c) for c in common_remove]
data_leaks = [lemmatizer.lemmatize(d) for d in data_leaks]
# due to using bigrams not including 
useless_singles=['','black','white','red','yellow','seed','breast','confectioner','sundried','broth','bell','baby','juice','crumb','sauce','condensed','smoked','basmati','extravirgin','brown','clarified', 'soy', 'filling', 'pine', 'virgin', 'romano', 'heart', 'shell', 'thigh', 'boneless','skinless','split', 'dark', 'wheat', 'light', 'green', 'vegetable', 'curry', 'orange', 'garam', 'sesame', 'strip', 'sea', 'canola', 'mustard','powder', 'ice', 'bay', 'roasted', 'loaf', 'roast', 'powder']
useless_singles = [lemmatizer.lemmatize(u) for u in useless_singles]

[nltk_data] Downloading package wordnet to /home/l/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



## Create SKLearn Pipeline 
Here I put together the pipeline, using the class weighted Logistic Regression model. Additionally, the TransformRecipe class I wrote above is used here as part of the pipeline. It was easier for me to work with the data as a list of lists instead of a pandas dataframe, I can use a list or list of lists as input to the API, and since it works, I won't change it now.

In [18]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,StratifiedKFold, train_test_split
import pprint
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

skf=StratifiedKFold(n_splits=3)
pp = pprint.PrettyPrinter(indent=4)


print("\nresults for LR count vector")
lr_pipe = Pipeline([('parse_recipe_text', TransformRecipe()),
    ('vect', CountVectorizer(ngram_range=(1, 2))), 
    ('lr', LogisticRegression( max_iter=1000,random_state=123, 
    class_weight='balanced',multi_class='multinomial', solver='lbfgs'))])


cv=cross_validate(lr_pipe, df['ingredients'], df['cuisine'].values, scoring='f1_weighted', 
                         cv=skf, return_train_score=True )

pp.pprint(cv)

X_train, X_test, y_train, y_test = train_test_split(df['ingredients'].values, df['cuisine'], test_size=0.25, stratify=df['cuisine'].tolist())

lr_pipe.fit(X_train, y_train)

# #Pickle pipeline after caling the fit
from sklearn.externals import joblib
joblib.dump(lr_pipe, 'lr_pipe.pkl')



results for LR count vector


/home/l/anaconda3/envs/datascience/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/l/anaconda3/envs/datascience/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/l/anaconda3/envs/datascience/lib/python3.6/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/l/anaconda3/envs/datascience/lib/python3.6/site-packages/sklearn/pipeline.py", line 307, in _fit
    **fit_params_steps[name])
  File "/home/l/anaconda3/envs/datascience/lib/python3.6/site-packages/joblib/memory.py", line 352, in __call__
    return self.func(*args, **kwargs)
  File "/home/l/anaconda3/envs/datascience/lib/python3.6/site-packages/sklearn/pipeline.py",

{   'fit_time': array([0.94669032, 0.91727138, 0.93845272]),
    'score_time': array([0., 0., 0.]),
    'test_score': array([nan, nan, nan]),
    'train_score': array([nan, nan, nan])}


ValueError: empty vocabulary; perhaps the documents only contain stop words